### I/O Benchmarks

In [261]:
!rm -rf test*

In [262]:
!mkfile 500m test_500.txt ; mkfile 1024m test_1024.txt ; mkfile 2048m test_2048.txt

In [13]:
import io
import time
import itertools

In [14]:
global benchmarks

In [15]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            benchmarks[method.__name__].append((te - ts) * 1000)
        return result
    return timed

In [16]:
@timeit
def open_io(filename):
    b = bytearray(16) 
    f = io.open(filename, "rb")
    numread = f.readinto(b)
    file = f.read()
    f.close()
    return file

In [17]:
@timeit
def open_classic(filename):
    with open(filename) as file:
        data = file.read()
    return data

In [18]:
def run_bench(filename, try_cnt=10):
    global benchmarks
    benchmarks = {'open_io':[], "open_classic":[]}
    for i in range(0, try_cnt):
        open_io(filename)
        open_classic(filename)
    return benchmarks

In [21]:
try_cnt = 100
benchmarks = run_bench("test_2048.txt", try_cnt)

In [22]:
import plotly.graph_objects as go

x = [i for i in range(0, try_cnt)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=benchmarks['open_io'],
    name = "Open io and byte buffer [100m]",
))
fig.add_trace(go.Scatter(
    x=x,
    y=benchmarks['open_classic'],
    name = "Classic open [100m]",
))

fig.show()

In [23]:
from statistics import mean
mean(benchmarks['open_io']),mean(benchmarks['open_classic'])

(1886.0996055603027, 2086.7332530021667)